In [1]:
"""
Created on 24-April-2024
@author: jdmantillaq
"""

import plotly.express as px
import pandas as pd
import numpy as np
import dash
from dash import Dash, dcc, html, dash_table
import dash_bootstrap_components as dbc
from dash.dependencies import Output, Input, State
from dash.exceptions import PreventUpdate
from dash_bootstrap_templates import load_figure_template
import os
import requests
import dash_leaflet as dl

In [28]:
dbc_css = "https://cdn.jsdelivr.net/gh/AnnMarieW/dash-bootstrap-templates/dbc.min.css"
app = Dash(__name__, external_stylesheets=[dbc.themes.DARKLY, dbc_css])
app.title = 'Cambio Climático'

load_figure_template("darkly")
slider_style = {"font-size": "18px",
                'margin-top': '20px', 'left-right': '30px'}
text_style = {"font-size": "20px"}


map_dl = dl.Map([dl.TileLayer(),
                 dl.FullScreenControl(),
                 dl.ScaleControl(position="bottomleft")],
                zoom=4.5, center=(6.248011, -75.577277),
                style={'width': '100%',
                       'height': '75vh',
                       'margin': "auto",
                       "display": "block"})

# map_dl = html.Div()

# -----------------------------------------------------------------------------
# popover Datasets
# -----------------------------------------------------------------------------

models_NEX = ['ACCESS-CM2', 'ACCESS-ESM1-5', 'BCC-CSM2-MR', 'CanESM5',
              'CMCC-CM2-SR5', 'CMCC-ESM2', 'EC-Earth3',
              'EC-Earth3-Veg-LR', 'GFDL-CM4',
              'GFDL-ESM4', 'INM-CM4-8', 'INM-CM5-0',
              'IPSL-CM6A-LR', 'KACE-1-0-G', 'KIOST-ESM', 'MIROC6',
              'MPI-ESM1-2-HR', 'MPI-ESM1-2-LR', 'MRI-ESM2-0', 'NESM3',
              'NorESM2-LM', 'NorESM2-MM', 'TaiESM1']

models_NEX = ['BCC-CSM2-MR', 'MIROC6', 'MPI-ESM1-2-LR',  'MRI-ESM2-0',
              'NorESM2-LM']


models_HighResp = ['FGOALS-f3-L', 'MRI-AGCM3-2-H',
                   'MRI-AGCM3-2-S', 'NICAM16-7S', 'NICAM16-8S']

obs_list = ['ERA5', 'CHRIPS']

radioitems_dataset_obs = dbc.CardBody(dcc.RadioItems(obs_list,
                                                     id="radio_dataset_obs",
                                                     value=None),
                                      style=slider_style)


radioitems_dataset_NEX = dbc.CardBody(dcc.RadioItems(models_NEX,
                                                     id="radio_dataset_NEX",),
                                      style=slider_style)

radioitems_dataset_HighResp = dbc.CardBody(
    dcc.RadioItems(models_HighResp,
                   id="radio_dataset_HighResp",),
    style=slider_style)


content_dataset = dbc.PopoverBody(
    children=[dbc.Row([dbc.Col([html.H5(children="Observaciones")], width=4),
                       dbc.Col([html.H5(children="NEX-GDDP-CMIP6")], width=4),
                       dbc.Col([html.H5(children="HighResMIP-CMIP6")], width=4),
                       ]),
              html.Hr(),
              dbc.Row([dbc.Col([radioitems_dataset_obs], width=4),
                       dbc.Col([radioitems_dataset_NEX], width=4),
                       dbc.Col([radioitems_dataset_HighResp], width=4),
                       ])])


popovers_dataset = html.Div(
    [
        dbc.Button(
            "Dataset",
            id="button_dataset",
            n_clicks=0,
            style=slider_style
        ),
        dbc.Popover(
            [
                # dbc.PopoverHeader("Popover header"),
                dbc.PopoverBody(content_dataset),
            ],
            target="button_dataset",
            placement='bottom',
            trigger="legacy",
            autohide=True,
            style={'maxWidth': '3000px'},
            # body=True,
        ),
    ]
)

# -----------------------------------------------------------------------------
# popover extreme indexes
# -----------------------------------------------------------------------------
tas_index_list = ['TXx', 'TNn', 'TN10p', 'TN90p', 'TX10p', 'TX90p', 'DTR',
                  'WSDI']

pr_index_list = ['R5mm', 'R10mm', 'R20mm', 'R50mm', 'CDD', 'CWD', 'R95p',
                 'R99p', 'PRCPTOT', 'SDII', 'P75y', 'P90y', 'P95y',
                 'P75p', 'P90p', 'P95p']

radioitems_var_tas = dbc.CardBody(dcc.RadioItems(tas_index_list,
                                                 id="radio_var_tas",
                                                 value=None),
                                  style=slider_style)
radioitems_var_pr = dbc.CardBody(dcc.RadioItems(pr_index_list,
                                                id="radio_var_pr",),
                                 style=slider_style)


content_variable = dbc.PopoverBody(
    style={
        "display": "flex",
        "flexDirection": "row",
        "justifyContent": "space-between"
    },
    children=[
        dbc.Col([html.H5(children="Temperatura"),
                 html.Hr(),
                 radioitems_var_tas], width=7),
        dbc.Col([html.H5(children="Precipitación"),
                 html.Hr(),
                 radioitems_var_pr], width=7),
    ]
)


popovers_variable = html.Div(
    [
        dbc.Button(
            "Indices de Extremos",
            id="button_variable",
            n_clicks=0,
            style=slider_style
        ),
        dbc.Popover(
            [
                # dbc.PopoverHeader("Popover header"),
                dbc.PopoverBody(content_variable,
                                ),
            ],
            target="button_variable",
            placement='bottom',
            trigger="legacy",
            autohide=True,
            style={'maxWidth': '500px'},
            body=True,
        ),
    ]
)



# -----------------------------------------------------------------------------
# popover time period
# -----------------------------------------------------------------------------

hist_time_list = ['1979(81)-2014']

hist_time_proj = ['2021-2040', '2041-2060', '2061-2080', '2081-2100']

radioitems_time_hist = dbc.CardBody(dcc.RadioItems(hist_time_list,
                                                 id="radio_time_hist",
                                                 value=None),
                                  style=slider_style)
radioitems_time_proj= dbc.CardBody(dcc.RadioItems(hist_time_proj,
                                                id="radio_time_proj",),
                                 style=slider_style)


content_time_period = dbc.PopoverBody(
    style={
        "display": "flex",
        "flexDirection": "row",
        "justifyContent": "space-between"
    },
    children=[
        dbc.Col([html.H5(children="Histórico"),
                 html.Hr(),
                 radioitems_time_hist], width=8),
        dbc.Col([html.H5(children="Proyecciones"),
                 html.Hr(),
                 radioitems_time_proj], width=6),
    ]
)


# content_time_period = dbc.PopoverBody(
#     children=[dbc.Row([dbc.Col([html.H5(children="Histórico")], width=7),
#                        dbc.Col([html.H5(children="Proyecciones")], width=5),
#                        ]),
#               html.Hr(),
#               dbc.Row([dbc.Col([radioitems_time_hist], width=7),
#                        dbc.Col([radioitems_time_project], width=5),
#                        ])])


popovers_time_period = html.Div(
    [
        dbc.Button(
            "Periodo de estudio",
            id="button_time_period",
            n_clicks=0,
            style=slider_style
        ),
        dbc.Popover(
            [
                # dbc.PopoverHeader("Popover header"),
                dbc.PopoverBody(content_time_period,
                                ),
            ],
            target="button_time_period",
            placement='bottom',
            trigger="legacy",
            autohide=True,
            style={'maxWidth': '500px'},
            body=True,
        ),
    ]
)



# -----------------------------------------------------------------------------
# Tab 1: content
# -----------------------------------------------------------------------------


tab1_content = dbc.Card(
    dbc.CardBody(
        [dbc.Row([dbc.Col([popovers_dataset, popovers_variable, 
                           popovers_time_period],
                          style={"display": "flex", "flexWrap": "wrap"})]),
         dbc.Row(map_dl)]
    ))


# -----------------------------------------------------------------------------
# Tab 1: content
# -----------------------------------------------------------------------------


tab2_content = dbc.Card()


app.layout = html.Div(
    [html.Div(
        className="header",
        children=[
            html.Div(
                className="div-info",
                children=[
                    html.H4(className="title",
                            children="Cambio Climático",
                            style={"padding": "10px"}),
                    # html.Hr(),
                    dbc.Tabs([
                        dbc.Tab(tab1_content, label="Índices de Extremos",
                                tab_id="tab-1")],
                             id="tabs",
                             active_tab="tab-1",
                             )
                ],
            ),
        ],
    ),
    ], style=text_style
)


# -----------------------------------------------------------------------------
# Dataset
# -----------------------------------------------------------------------------
@app.callback(
    [Output("radio_dataset_obs", "value"),
     Output("radio_dataset_NEX", "value"),
     Output("radio_dataset_HighResp", "value")],
    [Input("radio_dataset_obs", "value"),
     Input("radio_dataset_NEX", "value"),
     Input("radio_dataset_HighResp", "value")]
)
def clear_selections_dataset(obs_value, nex_value, high_value):
    ctx = dash.callback_context
    if not ctx.triggered:
        return [None, None, None]
    else:
        prop_id = ctx.triggered[0]['prop_id'].split('.')[0]
        if prop_id == 'radio_dataset_obs':
            return [obs_value, None, None]
        elif prop_id == 'radio_dataset_NEX':
            return [None, nex_value, None]
        elif prop_id == 'radio_dataset_HighResp':
            return [None, None, high_value]
        else:
            return [None, None, None]

# -----------------------------------------------------------------------------
# Indices
# -----------------------------------------------------------------------------
@app.callback(
    [Output("radio_var_tas", "value"),
     Output("radio_var_pr", "value"),
     ],
    [Input("radio_var_tas", "value"),
     Input("radio_var_pr", "value"),
     ]
)
def clear_selections_var(value_1, value_2):
    ctx = dash.callback_context
    if not ctx.triggered:
        return [None, None]
    else:
        prop_id = ctx.triggered[0]['prop_id'].split('.')[0]
        if prop_id == 'radio_var_tas':
            return [value_1, None]
        elif prop_id == 'radio_var_pr':
            return [None, value_2]
        else:
            return [None, None]
        
        
# -----------------------------------------------------------------------------
# Time Period
# -----------------------------------------------------------------------------
@app.callback(
    [Output("radio_time_hist", "value"),
     Output("radio_time_proj", "value"),
     ],
    [Input("radio_time_hist", "value"),
     Input("radio_time_proj", "value"),
     ]
)
def clear_selections_time(value_1, value_2):
    ctx = dash.callback_context
    if not ctx.triggered:
        return [None, None]
    else:
        prop_id = ctx.triggered[0]['prop_id'].split('.')[0]
        if prop_id == 'radio_time_hist':
            return [value_1, None]
        elif prop_id == 'radio_time_proj':
            return [None, value_2]
        else:
            return [None, None]


app.run_server(port=2100, host='0.0.0.0')

In [84]:
import seaborn as sns
index_prop = {
        'R5mm': [np.linspace(0, 250), np.linspace(-30, 30),
                 'viridis', 'días/año'],
        'R10mm': [np.linspace(0, 200), np.linspace(-30, 30),
                  'viridis', 'días/año'],
        'R20mm': [np.linspace(0, 250), np.linspace(-30, 30),
                  'viridis', 'días/año'],
        'R50mm': [np.linspace(0, 250), np.linspace(-30, 30),
                  'viridis', 'días/año'],
        'CDD': [np.linspace(0, 60), np.linspace(-10, 10),
                'plasma', 'días/año'],
        'CWD': [np.linspace(0, 60), np.linspace(-10, 10),
                'plasma', 'días/año'],

        'R95p': [np.linspace(0, 1000), np.linspace(-350, 350),
                 sns.color_palette("Spectral_r", as_cmap=True),
                 'mm/año'],
        'R99p': [np.linspace(0, 400), np.linspace(-250, 250),
                 sns.color_palette("Spectral_r", as_cmap=True),
                 'mm/año'],
        'PRCPTOT': [np.linspace(0, 6000), np.linspace(-900, 900),
                    sns.color_palette("Spectral_r", as_cmap=True),
                    'mm/año'],

        'SDII': [np.linspace(0, 30), np.linspace(-5, 5),
                 sns.color_palette("Spectral_r", as_cmap=True),
                 'mm/día'],

        'P75y': [np.linspace(0, 60), np.linspace(-8, 8),
                 sns.color_palette("Spectral_r", as_cmap=True), 'mm/día'],
        'P90y': [np.linspace(0, 60), np.linspace(-8, 8),
                 sns.color_palette("Spectral_r", as_cmap=True), 'mm/día'],
        'P95y': [np.linspace(0, 60), np.linspace(-8, 8),
                 sns.color_palette("Spectral_r", as_cmap=True), 'mm/día'],
        
        'P75p': [np.linspace(0, 30), np.linspace(-8, 8),
                 sns.color_palette("Spectral_r", as_cmap=True), 'días/año'],
        'P90p': [np.linspace(0, 30), np.linspace(-8, 8),
                 sns.color_palette("Spectral_r", as_cmap=True), 'días/año'],
        'P95p': [np.linspace(0, 30), np.linspace(-8, 8),
                 sns.color_palette("Spectral_r", as_cmap=True), 'días/año'],
    }

In [85]:
index_prop.keys()

dict_keys(['R5mm', 'R10mm', 'R20mm', 'R50mm', 'CDD', 'CWD', 'R95p', 'R99p', 'PRCPTOT', 'SDII', 'P75y', 'P90y', 'P95y', 'P75p', 'P90p', 'P95p'])

In [87]:
index_prop = {
        'TXx': [np.linspace(18, 42), np.linspace(-6, 6),
                sns.color_palette("Spectral_r", as_cmap=True), '°C'],
        'TNn': [np.linspace(6, 32), np.linspace(-6, 6),
                sns.color_palette("Spectral_r", as_cmap=True), '°C'],

        'TN10p': [np.linspace(0, 15), np.linspace(-10, 10),
                  sns.color_palette("viridis", as_cmap=True),
                  '% días/año'],
        'TN90p': [np.linspace(0, 15), np.linspace(-80, 80),
                  sns.color_palette("viridis", as_cmap=True),
                  '% días/año'],

        'TX10p': [np.linspace(6, 15), np.linspace(-15, 15),
                  sns.color_palette("viridis", as_cmap=True),
                  '% días/año'],
        'TX90p': [np.linspace(6, 15), np.linspace(-90, 90),
                  sns.color_palette("viridis", as_cmap=True),
                  '% días/año'],

        'DTR': [np.linspace(6, 18), np.linspace(-1, 1),
                sns.color_palette("Spectral_r", as_cmap=True), '°C'],

        'WSDI': [np.linspace(0, 30), np.linspace(-250, 250),
                 sns.color_palette("plasma", as_cmap=True), 'días/año'],

    }

index_prop.keys()

dict_keys(['TXx', 'TNn', 'TN10p', 'TN90p', 'TX10p', 'TX90p', 'DTR', 'WSDI'])